In [1]:
import os, sys
print(os.getcwd())
print(sys.version)

/Users/cuong/Desktop/cs50/SGR/SGR_database/excel_to_db
3.9.5 (default, May  4 2021, 03:36:27) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
import pandas as pd
import numpy as np

In [4]:
vs = pd.read_excel("./industry_classification.xlsx", header=0, index_col=0, sheet_name="Danh sách tất cả công ty")
vs.head()

,Mã,Tên công ty,Phân ngành 1,Phân ngành 2,Sàn,Khối lượng niêm yết
STT,,,,,,
1,A32,CTCP 32,Sản xuất,Sản xuất các sản phẩm da và liên quan,UPCoM,6800000
2,AAA,CTCP Nhựa An Phát Xanh,Sản xuất,Sản xuất các sản phẩm nhựa và cao su,HOSE,221759974
3,AAM,CTCP Thủy sản MeKong,Sản xuất,Sản xuất thực phẩm,HOSE,12346411
4,AAT,CTCP Tiên Sơn Thanh Hóa,Bán buôn,Bán buôn hàng tiêu dùng,HOSE,34800000
5,AAV,CTCP Việt Tiên Sơn Địa ốc,Bán buôn,Bán buôn hàng tiêu dùng,HNX,31874996


In [6]:
fiin = pd.read_excel(r"./fiin_industry_classification.xlsx", header=8, index_col=0, sheet_name="ExportData")
fiin.head()

,Ticker,OrganShortName,ComGroupCode
1,BSR,Lọc - Hóa dầu Bình Sơn,UPCOM
2,OIL,PV Oil,UPCOM
3,PEQ,Thiết bị xăng dầu Petrolimex,UPCOM
4,PETROVN,PETROVIETNAM,OTC
5,PLX,Petrolimex,HOSE


In [7]:
fiin.drop(fiin.tail(15).index, inplace=True)
fiin

,Ticker,OrganShortName,ComGroupCode
1,BSR,Lọc - Hóa dầu Bình Sơn,UPCOM
2,OIL,PV Oil,UPCOM
3,PEQ,Thiết bị xăng dầu Petrolimex,UPCOM
4,PETROVN,PETROVIETNAM,OTC
5,PLX,Petrolimex,HOSE
...,...,...,...
3278,VIE,CN Viễn thông VI TE CO,HNX
3279,VJTC,Thương mại Việt Nhật,OTC
3280,VLA,PT Công nghệ Văn Lang,HNX
3281,VTC,Viễn thông VTC,HNX


In [26]:
fiin_tickers = set(fiin['Ticker'].values)
vs_tickers = set(vs['Mã'].values)

In [28]:
fiin_difference = list(fiin_tickers - vs_tickers)
len(fiin_difference)

1314

In [29]:
vs_difference = list(vs_tickers - fiin_tickers)
len(vs_difference)

963

In [30]:
shared = list(vs_tickers & fiin_tickers)
len(shared)

1968

In [39]:
shared_filt = fiin['Ticker'].isin(shared)
fiin.loc[shared_filt, 'ComGroupCode'].value_counts()

UPCOM    934
HOSE     394
HNX      360
OTC      280
Name: ComGroupCode, dtype: int64

In [40]:
vs_diff_filt = vs['Mã'].isin(vs_difference)
vs.loc[vs_diff_filt, 'Sàn'].value_counts()

OTC          776
Khác         176
Campuchia      6
Lào            5
Name: Sàn, dtype: int64

In [38]:
fiin_diff_filt = fiin['Ticker'].isin(fiin_difference)
fiin.loc[fiin_diff_filt, 'ComGroupCode'].value_counts()

OTC      1249
UPCOM      56
HNX         6
HOSE        3
Name: ComGroupCode, dtype: int64

In [46]:
fiin_diff_filt = fiin['Ticker'].isin(fiin_difference)
fiin_exchange_filt = fiin['ComGroupCode'] == 'OTC'
fiin.loc[fiin_diff_filt & fiin_exchange_filt, ['OrganShortName', 'Ticker']]

,OrganShortName,Ticker
4,PETROVIETNAM,PETROVN
15,Kỹ nghệ nội thất ACE,ACEW
22,Hóa chất Á Châu,ASIAC
25,Bao Bì Dược,BAOBIDUOC
29,Khoáng sản Biotan,BIOTAN
...,...,...
3260,Kim tự tháp Việt Nam,PYTHIS
3270,Thiết bị bưu điện,TFPC
3272,Cơ điện Trần Phú,TPU
3276,Cáp quang Việt Đức,VDFC


In [49]:
fiin_diff_filt = fiin['Ticker'].isin(fiin_difference)
fiin_diff = list(fiin.loc[fiin_diff_filt, 'OrganShortName'].values)
vs_diff_filt = vs['Mã'].isin(vs_difference)
vs_diff = list(vs.loc[vs_diff_filt, 'Tên công ty'].values)

In [52]:
import re
fiin_dict = dict()
for fiin_var in fiin_diff:
    fiin_dict[fiin_var] = []
    for vs_var in vs_diff:
        q = re.compile(r'.*{}.*'.format(fiin_var), re.IGNORECASE)
        m = q.match(vs_var)
        if m: 
            fiin_dict[fiin_var].append(m.group())

In [54]:
count = 0
for key, value in fiin_dict.items():
    if value:
        count += 1
print(count)

345


In [57]:
count = 0
check_dict = dict()
for key, value in fiin_dict.items():
    if len(value) > 1:
        check_dict[key] = value

In [58]:
check_dict

{'CTCP Hóa chất': ['CTCP Hóa chất Á Châu',
  'CTCP Hóa chất và Vật tư Khoa học Kỹ thuật',
  'CTCP Hóa Chất',
  'CTCP Hóa chất Miền Nam',
  'CTCP Hóa chất Minh Đức'],
 'VINACOMIN': ['CTCP Cơ khí Hòn Gai - Vinacomin',
  'CTCP Du lịch và Thương mại Vinacomin',
  'CTCP Than Miền Nam - Vinacomin',
  'CTCP Than Cao Sơn - Vinacomin',
  'CTCP Than Tây Nam Đá mài - Vinacomin',
  'Công ty TNHH MTV Đầu tư Phát triển Nhà và Hạ Tầng - Vinacomin'],
 'CIENCO 1': ['CTCP Cầu 14 - Cienco 1',
  'CTCP Xây dựng và Đầu tư 122 - Cienco 1'],
 'COSEVCO': ['CTCP Sản xuất Vật liệu và Xây dựng Cosevco 1',
  'CTCP Cosevco 75'],
 'Xi măng Hà Tiên': ['CTCP Xi măng Hà Tiên Kiên Giang',
  'CTCP Xi măng Hà Tiên 2'],
 'Phát triển Khoáng Sản': ['CTCP Phát triển Khoáng sản 4',
  'CTCP Phát triển khoáng Sản'],
 'Xây dựng Tây Hồ': ['CTCP Đầu Tư và Xây dựng Tây Hồ',
  'CTCP Đầu tư Phát triển Nhà và Xây dựng Tây Hồ'],
 'Trường Sơn': ['CTCP Trường Sơn Hà Nội',
  'CTCP Trường Sơn',
  'CTCP Chứng khoán Trường Sơn'],
 'Mía đườn

In [82]:
fiin_recheck_filt = fiin['OrganShortName'].isin(list(check_dict.keys()))
fiin_go_check = fiin.loc[fiin_recheck_filt, ['OrganShortName', 'Ticker']]
fiin_go_check


,OrganShortName,Ticker
45,CTCP Hóa chất,CHEMCO
328,VINACOMIN,VNCOMIN
501,CIENCO 1,CIEN1
536,COSEVCO,COSEV
778,Xi măng Hà Tiên,HTCJ
911,Phát triển Khoáng Sản,MEDIC
1246,Xây dựng Tây Hồ,THHOUS
1291,Trường Sơn,TSJC
1853,Mía đường I,SUGAR1
1868,Thanh Bình,TBCORP
